In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
def read_data(file_path):
    tokens = []
    tags = []
    
    tweet_tokens = []
    tweet_tags = []
    for line in open(file_path, encoding='utf-8'):
        line = line.strip()
        if not line:
            if tweet_tokens:
                tokens.append(tweet_tokens)
                tags.append(tweet_tags)
            tweet_tokens = []
            tweet_tags = []
        else:
            token, tag = line.split()
            if token.startswith("@"):
                token="<USR>"
            elif token.startswith("http://") or token.startswith("https://"):
                token="<URL>"
            
            tweet_tokens.append(token)
            tweet_tags.append(tag)
            
    return tokens, tags

In [4]:
X_train, y_train = read_data('data/train.txt')
X_val, y_val = read_data('data/validation.txt')
X_test, y_test = read_data('data/test.txt')

In [18]:
wordsets={}
X=X_train
for x in X:
    for word in x:
        wordsets[word]=1
X=X_val
for x in X:
    for word in x:
        wordsets[word]=1
X=X_test
for x in X:
    for word in x:
        wordsets[word]=1
wordsets=list(wordsets.keys())
NX=len(wordsets)
wordsets={}
y_train1=y_train
for tag in y_train1:
    for word in tag:
        wordsets[word]=1
y_train1=y_val
for tag in y_train1:
    for word in tag:
        wordsets[word]=1
Ny=len(wordsets.keys())

In [21]:
X=X_train+X_val+X_test
ml=max([len(x) for x in X])

In [22]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer=Tokenizer(num_words=NX)
tokenizer.fit_on_texts(list(X_train)+list(X_val)+list(X_test))
sequences=tokenizer.texts_to_sequences(list(X_train)+list(X_val))
X_new=pad_sequences(sequences,maxlen=ml)

In [ ]:
trbl=10
num_layers=3
state_size=128
num_classes=100
hidden_size=100
p=0.5
num_features=100
max_length=max(max([len(x) for x in X_train]),max([len(x) for x in X_val]),max([len(x) for x in X_test]))
batch_size=100
num_epochs=1000
batches=1000

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer2=Tokenizer(num_words=Ny)
tokenizer2.fit_on_texts(list(y_train)+list(y_val))
sequences2=tokenizer2.texts_to_sequences(list(y_train)+list(y_val))
labels=[]
y=list(y_train)+list(y_val)
for tag in sequences2:
    t=np.zeros(Ny)
    for wordid in tag:
        t[sequences2[wordid]]=1
    labels.append(t)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer=Tokenizer(num_words=NX+1)
tokenizer.fit_on_texts(list(X_train)+list(X_val)+list(X_test))
sequences=tokenizer.texts_to_sequences(list(X_train)+list(X_val))
ml=max([len(x) for x in sequences])
X_train_new=pad_sequences(sequences,maxlen=ml)
model=Sequential()
model.add(Embedding(input_dim=NX,output_dim=num_features,input_length=ml,trainable=True,mask_zero=0))
model.add(LSTM(units=state_size,dropout=0.5,recurrent_dropout=0.5))
model.add(Dense(num_classes,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X_train_new,np.array(labels),validation_split=0.2,epochs=10)